In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/whisper-models/medium.en (1).pt
/kaggle/input/whisper-models/small.en (2).pt
/kaggle/input/roberta-base-v1/config.json
/kaggle/input/roberta-base-v1/merges.txt
/kaggle/input/roberta-base-v1/tokenizer.json
/kaggle/input/roberta-base-v1/vocab.json
/kaggle/input/roberta-base-v1/tokenizer_config.json
/kaggle/input/roberta-base-v1/model.safetensors
/kaggle/input/roberta-base-v1/special_tokens_map.json
/kaggle/input/roberta-base-v1/.virtual_documents/__notebook_source__.ipynb
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_49.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_67_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_90.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_77.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_20_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_66.wav
/kaggle/input/shl-int

In [2]:
pip install -U openai-whisper


Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install -q protobuf==3.20.3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, bu

In [3]:
# ============================================================
# 0. IMPORTS
# ============================================================
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from tqdm.auto import tqdm
import whisper
import gc
import os

# ============================================================
# 1. CONFIGURATION
# ============================================================
MODEL_PATH = "/kaggle/input/roberta-base-v1"

# 🔹 Whisper .pt files (CORRECT)
PATH_WHISPER_SMALL  = "/kaggle/input/whisper-models/small.en (2).pt"
PATH_WHISPER_MEDIUM = "/kaggle/input/whisper-models/medium.en (1).pt"

BASE_DIR = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset"

BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1.5e-5
MAX_LEN = 512
SEED = 42

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# 2. LOAD CSVs
# ============================================================
train_df = pd.read_csv(f"{BASE_DIR}/csvs/train.csv")
test_df  = pd.read_csv(f"{BASE_DIR}/csvs/test.csv")

print("✅ CSVs loaded")

# ============================================================
# 3. WHISPER TRANSCRIPTION FUNCTION
# ============================================================
def run_whisper_on_dataframe(df, model_path, col_name, split):
    print(f"\n⏳ Loading Whisper model: {model_path}")
    model = whisper.load_model(model_path)

    def _transcribe(filename):
        audio_path = f"{BASE_DIR}/audios/{split}/{filename}.wav"
        if not os.path.exists(audio_path):
            return ""
        try:
            return model.transcribe(audio_path)["text"].strip()
        except:
            return ""

    tqdm.pandas()
    print(f"🗣️ Transcribing {len(df)} files → {col_name}")
    df[col_name] = df["filename"].progress_apply(_transcribe)

    del model
    gc.collect()
    torch.cuda.empty_cache()
    return df

# ============================================================
# 4. EXECUTE TRANSCRIPTION
# ============================================================
# TEST
test_df  = run_whisper_on_dataframe(test_df,  PATH_WHISPER_SMALL,  "text_small",  "test")
test_df  = run_whisper_on_dataframe(test_df,  PATH_WHISPER_MEDIUM, "text_medium", "test")

# TRAIN
train_df = run_whisper_on_dataframe(train_df, PATH_WHISPER_SMALL,  "text_small",  "train")
train_df = run_whisper_on_dataframe(train_df, PATH_WHISPER_MEDIUM, "text_medium", "train")

# SAFETY
for df in [train_df, test_df]:
    df["text_small"]  = df.get("text_small", "").fillna("")
    df["text_medium"] = df.get("text_medium", df["text_small"]).fillna("")

print("✅ Transcription completed")

# ============================================================
# 5. DATASET
# ============================================================
class StereoDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len=MAX_LEN, is_test=False):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row["text_small"] + " </s> " + row["text_medium"]
        label = 0.0 if self.is_test else float(row.get("label", 0.0))

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt",
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.float),
        }

# ============================================================
# 6. MODEL
# ============================================================
class CustomModel(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.base = AutoModel.from_pretrained(model_path)
        self.fc = nn.Linear(self.base.config.hidden_size, 1)

    def forward(self, inputs):
        out = self.base(**inputs).last_hidden_state
        mask = inputs["attention_mask"].unsqueeze(-1).float()
        pooled = (out * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
        return self.fc(pooled)

# ============================================================
# 7. TRAINING (5-FOLD CV)
# ============================================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
final_preds = np.zeros(len(test_df))

print("\n🚀 Training started")

for fold, (tr_idx, _) in enumerate(kf.split(train_df)):
    print(f"\n--- Fold {fold + 1} ---")

    train_fold = train_df.iloc[tr_idx]

    train_ds = StereoDataset(train_fold, tokenizer)
    test_ds  = StereoDataset(test_df, tokenizer, is_test=True)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE)

    model = CustomModel(MODEL_PATH).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    sch = get_linear_schedule_with_warmup(opt, 0, len(train_loader) * EPOCHS)
    loss_fn = nn.MSELoss()

    model.train()
    for _ in range(EPOCHS):
        for b in train_loader:
            inp = {
                "input_ids": b["input_ids"].to(DEVICE),
                "attention_mask": b["attention_mask"].to(DEVICE),
            }
            y = b["labels"].to(DEVICE)

            opt.zero_grad()
            loss = loss_fn(model(inp).view(-1), y)
            loss.backward()
            opt.step()
            sch.step()

    model.eval()
    preds = []
    with torch.no_grad():
        for b in test_loader:
            inp = {
                "input_ids": b["input_ids"].to(DEVICE),
                "attention_mask": b["attention_mask"].to(DEVICE),
            }
            preds.append(model(inp).view(-1).cpu().numpy())

    final_preds += np.concatenate(preds) / 5

    del model, opt, sch
    gc.collect()
    torch.cuda.empty_cache()

# ============================================================
# 8. SUBMISSION
# ============================================================
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": final_preds.clip(0, 5)
})

submission.to_csv("submission_stereo_roberta.csv", index=False)
print("\n🏆 submission_stereo_roberta.csv created")

✅ CSVs loaded

⏳ Loading Whisper model: /kaggle/input/whisper-models/small.en (2).pt
🗣️ Transcribing 197 files → text_small


  0%|          | 0/197 [00:00<?, ?it/s]


⏳ Loading Whisper model: /kaggle/input/whisper-models/medium.en (1).pt
🗣️ Transcribing 197 files → text_medium


  0%|          | 0/197 [00:00<?, ?it/s]


⏳ Loading Whisper model: /kaggle/input/whisper-models/small.en (2).pt
🗣️ Transcribing 409 files → text_small


...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'


--- Fold 2 ---

--- Fold 3 ---

--- Fold 4 ---

--- Fold 5 ---

🏆 submission_stereo_roberta.csv created
